### Pull Full Text Papers ###

In [1]:
import pandas as pd
# from metapub import PubMedFetcher
# import requests
# from semanticscholar import SemanticScholar
# import urllib
from pathlib import Path
import duckdb
import chardet
import numpy as np
import re

from dotenv import dotenv_values
config = dotenv_values('../../.env-secrets')
from glob import glob
import os
import re

In [ ]:
cwd = Path(os.getcwd())
root_dir_name = "therapeutic_accelerator"
root_dir_path = Path(
    *cwd.parts[: cwd.parts.index("therapeutic_accelerator") + 1])
config = dotenv_values(str(Path(root_dir_path, ".env-secrets")))

In [ ]:
def isEnglish(s):
    try:
        s.encode(encoding='ascii')
    except UnicodeEncodeError:
        return False
    else:
        return True

def extract_field_of_study(x): 
    if isinstance(x, np.ndarray): 
        return list(np.unique([i['category'] for i in x]))
    else: 
        return x
def load_df(file_path): 
    df = duckdb.sql(f"select * from '{file_path}'").fetchdf()
    return df

def cleaning_df(df): 
    df = df.drop(columns=['publicationvenueid'])
    df = df[df['isopenaccess']]
    df['is_english'] = df['title'].apply(isEnglish)
    df = df[df['is_english']] # assumption based on encoding
    df['title'] = df['title'].str.replace("\u2013", ' ')
    df['year'] = df['year'].fillna('').astype(str).str.strip('0').str.strip('.')
    df['eternalids'] = df['eternalids'].apply(lambda y: {k:v for k,v in y.items() if v})
    
    return df

def write_out_df(df, file_path): 
    parquet_file = re.sub('|'.join(Path(file_path).suffixes), '', str(Path(file_path)))  + '.parquet.gzip'
    print(parquet_file)
    df.to_parquet(parquet_file)
    

### Parse Files ###

**Import List of Corpus ids**

In [2]:
# df = pd.read_csv('final_corpuslist.csv')

# t = tuple(df['corpusid'])
# query = "select * from article_attributes where corpusid IN {}".format(t)

In [3]:
metadata_files = glob('../data/papers_metadata/*.gz', recursive=True)
fulltext_files = glob('../data/fulltext-zipped/*.gz', recursive=True)

In [65]:
x = duckdb.sql(f"select * from '{metadata_files[0]}'").fetchdf()
display(x.head())
print(x.shape)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,corpusid,externalids,url,title,authors,venue,publicationvenueid,year,referencecount,citationcount,influentialcitationcount,isopenaccess,s2fieldsofstudy,publicationtypes,publicationdate,journal
0,144907779,"{'ACL': None, 'DBLP': None, 'ArXiv': None, 'MA...",https://www.semanticscholar.org/paper/ef627608...,Modeling the Effects of Positive Affect on the...,"[{'authorId': '2115584035', 'name': 'Sungjin P...",,NaN,2014.0,28,2,0,False,"[{'category': 'Psychology', 'source': 's2-fos-...",[JournalArticle],2014-09-01,{'name': 'Proceedings of the Human Factors and...
1,234553368,"{'ACL': None, 'DBLP': None, 'ArXiv': None, 'MA...",https://www.semanticscholar.org/paper/6d895636...,Dakwah Media Sosial: Alternatif Dakwah Kontemp...,"[{'authorId': '2092150625', 'name': 'Iqrom Fal...",,NaN,2020.0,0,2,0,False,"[{'category': 'Sociology', 'source': 's2-fos-m...",NaN,2020-12-07,"{'name': '', 'pages': '36-58', 'volume': '15'}"
2,140925387,"{'ACL': None, 'DBLP': None, 'ArXiv': None, 'MA...",https://www.semanticscholar.org/paper/dd799468...,Razonamiento moral y prosocialidad: fundamentos,"[{'authorId': '38673449', 'name': 'Manuel Mart...",,NaN,2010.0,0,9,0,False,NaN,NaN,NaT,"{'name': '', 'pages': None, 'volume': ''}"
3,35822046,"{'ACL': None, 'DBLP': None, 'ArXiv': None, 'MA...",https://www.semanticscholar.org/paper/83ef5ee6...,Clinical outcomes of contrast-induced nephropa...,"[{'authorId': '51306154', 'name': 'Shao‐Liang ...",International Journal of Cardiology,a87fe009-4a3a-4b82-a6b0-3fb3a03a356c,2008.0,23,105,10,False,"[{'category': 'Medicine', 'source': 's2-fos-mo...","[Study, JournalArticle]",2008-06-06,{'name': 'International journal of cardiology'...
4,239051517,"{'ACL': None, 'DBLP': None, 'ArXiv': None, 'MA...",https://www.semanticscholar.org/paper/cc075a7f...,Use of Machine Learning-Based Software for the...,"[{'authorId': '2366754', 'name': 'D. Dov'}, {'...",Archives of Pathology & Laboratory Medicine,ebd38f08-32e6-40af-97cb-d1cac716e075,2021.0,0,7,1,False,"[{'category': 'Medicine', 'source': 's2-fos-mo...","[JournalArticle, Review]",2021-10-20,{'name': 'Archives of pathology & laboratory m...


(2272069, 16)


In [76]:
parquet_file = re.sub('|'.join(Path(metadata_files[0]).suffixes), '', str(Path(metadata_files[0])))  + '.parquet.gzip'
x.to_parquet(parquet_file)

In [79]:
import pyarrow.parquet as pq

In [80]:
test = pq.read_table(parquet_file)
test.shape

(221088, 17)

In [81]:
test

pyarrow.Table
corpusid: int64
externalids: struct<ACL: string, ArXiv: string, CorpusId: string, DBLP: string, DOI: string, MAG: string, PubMed: string, PubMedCentral: string>
  child 0, ACL: string
  child 1, ArXiv: string
  child 2, CorpusId: string
  child 3, DBLP: string
  child 4, DOI: string
  child 5, MAG: string
  child 6, PubMed: string
  child 7, PubMedCentral: string
url: string
title: string
authors: list<element: struct<authorId: string, name: string>>
  child 0, element: struct<authorId: string, name: string>
      child 0, authorId: string
      child 1, name: string
venue: string
year: string
referencecount: int64
citationcount: int64
influentialcitationcount: int64
isopenaccess: bool
s2fieldsofstudy: list<element: struct<category: string, source: string>>
  child 0, element: struct<category: string, source: string>
      child 0, category: string
      child 1, source: string
publicationtypes: list<element: string>
  child 0, element: string
publicationdate: timestamp[u

**Parse all CVS to get json into dataframe columns**

In [ ]:
import ast
from pandas.io.json import json_normalize
import time


def only_dict(d):
    return ast.literal_eval(d)


seconds = time.time()

for i in range(1,31):
    df = pd.read_csv(str(i) + '.csv')
    df = df[['corpusid','content']]
    A = json_normalize(df['content'].apply(only_dict).tolist())
    df = df[['corpusid']].join([A])
    df.to_csv('{}_full_text.csv'.format(i))

print("Seconds since epoch =", time.time() - seconds)



**Export into CSV and check final results**

In [ ]:
csv = []

for i in range(1,31):
    csv.append(str(i) + '_full_text.csv')
    
df = pd.concat(map(pd.read_csv, csv))


df.head(5)

In [ ]:
df1 = pd.read_csv('final_corpuslist.csv')

set(df['corpusid']).issubset(set(df1['corpusid']))